## 2nd assignment
Please submit your Python code and output in the following format:
Zone_id label (not coordinates), 1st nearest neighbor zone label (not coordinates) : distance (in miles), … , 6th nearest neighbor zone label (not coordinates) : distance (in miles).

In [101]:
import pandas as pd
import numpy as np
from scipy import spatial
import math

### DataSet Modified

In [102]:
#import coordinates.txt 
coor_df = pd.read_table('./coordinates.txt')
coor_df.shape## Final_Sheet

(1944, 1)

In [103]:
# making separate column from new data frame
new = coor_df["zone09  Latitude        Longitude"].str.split(" ", n = 1, expand = True)   
coor_df["zone09"]= new[0] 
coor_df["Latitude, Longitude"]= new[1]   
coor_df.drop(columns =["zone09  Latitude        Longitude"], inplace = True)  
coor_df.head()

,zone09,"Latitude, Longitude"
0,1,41.89457679280 -87.64530747770
1,2,41.89464672260 -87.64042628850
2,3,41.89471289300 -87.63551301540
3,4,41.89476770920 -87.63062889530
4,5,41.89483043620 -87.62579253650


In [104]:
#separate the Lat, Long column
new = coor_df["Latitude, Longitude"].str.split(" ") 
for i in range(len(new)):
    new[i]=list(filter(None, new[i]))
coor_table = pd.DataFrame(list(new),columns =['Latitude', 'Longitude'])
coor_table.shape

(1944, 2)

In [105]:
order= list(range(1,coor_table.shape[0]+1))
coor_table.insert(0, "zone09", order) 

In [106]:
coor_table.head()

,zone09,Latitude,Longitude
0,1,41.89457679280,-87.64530747770
1,2,41.89464672260,-87.64042628850
2,3,41.89471289300,-87.63551301540
3,4,41.89476770920,-87.63062889530
4,5,41.89483043620,-87.62579253650


In [108]:
coor_table = coor_table.astype(dtype= { "Latitude":"float64","Longitude":"float64"})
coor_table.dtypes

zone09         int64
Latitude     float64
Longitude    float64
dtype: object

### Find 6 Closest Zone

In [109]:
#Euclidian distance
def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = float(latitude) * (math.pi / 180)
    longitude = float(longitude) * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)

In [110]:
#We can use a k-dimensional tree to organize our points in our 3-dimensional space.
#This is a data structure from computer science that can help with searches over multidimensional keys

places = []
coor_array = np.array(coor_table)
for i in range(0, coor_array.shape[0]):
    coordinates = [coor_array[i][1],coor_array[i][2]]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

tree = spatial.KDTree(places)

In [111]:
tree.data

array([[  194.84468086, -4738.40722202,  4254.31234733],
       [  195.24814467, -4738.38541745,  4254.31813546],
       [  195.65427103, -4738.36374786,  4254.32361242],
       ...,
       [  106.23947066, -4689.90646633,  4310.81564347],
       [  120.39614374, -4691.53095374,  4308.6753276 ],
       [  133.57367267, -4691.1580953 ,  4308.69293392]])

In [112]:
def find_neighbor(lat, lon):
    cartesian_coord = cartesian(lat, lon)
    closest = tree.query([cartesian_coord],k=7,p = 2)   #return 7 closest neighbor and distance
    index = closest[1][0]
    return closest  

In [117]:
final_list=[]
for i in range(0, coor_array.shape[0]):
    sec = []
    sec.append(i+1)
    neighbor_list = find_neighbor ( coor_array[i][1], coor_array[i][2])   #give zone's latitude, longitude
    neighbor_zone = list(neighbor_list)[1]
    neighbor_distance = list(neighbor_list)[0]

    ratio = 0.621371
    for j in range(1,7):
        zone_id = coor_array[neighbor_zone[0][j]][0]
        distance = round(neighbor_distance[0][j]*ratio,4)   #distance in km -> miles
        zone_dis = (int(zone_id),distance)                  #(zone_label: distance) store in tuple
        sec.append(zone_dis)
    final_list.append(sec)

### Final_Sheet

In [120]:
# output final_list to final DataFrame
final = pd.DataFrame(final_list, columns = ['zone09','1st_nearest','2nd_nearest','3rd_nearest','4th_nearest',\
                                    '5th_nearest','6th_nearest'])

In [119]:
# export final DataFrame to excel_sheet
writer = pd.ExcelWriter('Assignment2.xlsx', engine='xlsxwriter')
final.to_excel(writer, sheet_name='Sheet1')
writer.save()